# Práctico 3: Números Aleatorios y Método de Monte Carlo

## Algoritmos

In [1]:
import math
import random as rnd

def isPrime(n: int) -> bool:
    """
    Check if a number is prime
    """
    if n == 1: return False
    elif n % 2 == 0: return n == 2
    
    i = 3
    while i*i <= n:
        if n % i == 0: return False
        i += 2
    return True

def factorize(n: int) -> dict:
    """
    Factorize a number into its prime factors
    """
    factors = dict()
    i = 2
    while i <= n:
        while n % i == 0:
            factors[i] = factors.get(i, 0) + 1
            n //= i
        i += 1
    return factors

def isPrimitiveRoot_forPrime(n: int, m: int) -> bool:
    """
    Check if n is a primitive root of m (with m prime)
    """
    assert isPrime(m)
    primes = factorize(m-1)
    for p in primes:
        if pow(n, (m-1)//p, m) == 1: return False
    return True

if __name__ == "__main__":
    numbers = [2, 15, 12_345, 2**7-1, 2**13-1]
    # Test the isPrime function
    for n in numbers:
        print(f"{n} is prime: {isPrime(n)}")

    # Test the factorize function
    for n in numbers:
        print(f"Factors of {n}: {factorize(n)}")
    
    # Test the isPrimitiveRoot_forPrime function
    true_cases = [
        [2, [1]],
        [3, [2]],
        [5, [2, 3]],
        [7, [3, 5]],
        [11, [2, 6, 7, 8]],
        [13, [2, 6, 7, 11]],
    ]
    for m, roots in true_cases:
        for root in roots:
            assert isPrimitiveRoot_forPrime(root, m)

    false_cases = [
        [3, [1]],
        [5, [1, 4]],
        [7, [1, 2, 4, 6]],
        [11, [1, 3, 4, 5, 9, 10]],
        [13, [1, 3, 4, 5, 9, 10, 12]],
    ]
    for m, roots in false_cases:
        for root in roots:
            assert not isPrimitiveRoot_forPrime(root, m)

2 is prime: True
15 is prime: False
12345 is prime: False
127 is prime: True
8191 is prime: True
Factors of 2: {2: 1}
Factors of 15: {3: 1, 5: 1}
Factors of 12345: {3: 1, 5: 1, 823: 1}
Factors of 127: {127: 1}
Factors of 8191: {8191: 1}


### Von Neumann

In [2]:
def vonNeumann(seed: int, n: int) -> list[int]:
    """
    Generate a list of n pseudo-random numbers using the Von Neumann method.
    """
    numbers = []
    for i in range(n):
        numbers.append(seed)
        seed = (seed ** 2 // 100) % 10_000
    return numbers

# Test the function with the example values
if __name__ == "__main__":
    n = 10
    seed_list = [4010, 2100, 3792, 1234]
    for seed in seed_list:
        print(f"n, seed = {n}, {seed}: {vonNeumann(seed, n)}")

n, seed = 10, 4010: [4010, 801, 6416, 1650, 7225, 2006, 240, 576, 3317, 24]
n, seed = 10, 2100: [2100, 4100, 8100, 6100, 2100, 4100, 8100, 6100, 2100, 4100]
n, seed = 10, 3792: [3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792]
n, seed = 10, 1234: [1234, 5227, 3215, 3362, 3030, 1809, 2724, 4201, 6484, 422]


### Generadores congruenciales lineales

In [3]:
def congruentialGenerator(seed: int, a: int, c: int, m: int, n: int) -> list[int]:
    """
    Generate a list of n pseudo-random numbers using the congruential generator method.
    """
    numbers = []
    for i in range(n):
        numbers.append(seed)
        seed = (a * seed + c) % m
    return numbers

def periodBrute_congruentialGenerator(seed: int, a: int, c: int, m: int) -> int:
    """
    Calculate the period of the congruential generator method with brute force.
    """
    periods = []
    seeds = range(m) if seed is None else [seed]
    for seed in seeds:
        numbers = dict()
        while seed not in numbers:
            numbers[seed] = len(numbers)
            seed = (a * seed + c) % m
        periods.append(len(numbers) - numbers[seed])
    return max(periods)

def period_congruentialGenerator(seed: int, a: int, c: int, m: int) -> int:
    """
    Calculate the period of the congruential generator method.
    """
    # If the generator is mixed and has the biggest period
    if c != 0 and math.gcd(c, m) == 1 and (a % p == 1 for p in factorize(m).keys()) and (m % 4 == 0) == (a % 4 == 1):
        return m
    
    # If the generator is multiplicative and has the biggest period
    if c == 0 and isPrime(m) and isPrimitiveRoot_forPrime(a, m):
        return m-1
    
    # Otherwise, calculate the period with brute force
    return periodBrute_congruentialGenerator(seed, a, c, m)

if __name__ == "__main__":
    # Test the congruentialGenerator function
    test_cases = [
        [0, 5, 1, 16, 20],
        [1, 1, 11, 22, 20],
        [0, 1, 1, 15, 20],

        # Mixed generator with biggest period
        [0, 5, 3, 16, 20],
        [0, 1_103_515_245, 12_345, 2**32, 20], # ANSI C
        
        # Multiplicative generator with biggest period
        [1, 2, 0, 19, 20],
        [1, 2**5, 0, 19, 20],
        [1, 7**5, 0, 2**31-1, 20],
    ]

    for test in test_cases:
        print(f"seed, a, c, m, n = {test[:4]}, {test[4]}: {congruentialGenerator(*test)}")
    
    # Test the period_congruentialGenerator function
    for test in test_cases:
        print(f"seed, a, c, m = {test[:4]}: {period_congruentialGenerator(*test[:4])}")    


seed, a, c, m, n = [0, 5, 1, 16], 20: [0, 1, 6, 15, 12, 13, 2, 11, 8, 9, 14, 7, 4, 5, 10, 3, 0, 1, 6, 15]
seed, a, c, m, n = [1, 1, 11, 22], 20: [1, 12, 1, 12, 1, 12, 1, 12, 1, 12, 1, 12, 1, 12, 1, 12, 1, 12, 1, 12]
seed, a, c, m, n = [0, 1, 1, 15], 20: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 1, 2, 3, 4]
seed, a, c, m, n = [0, 5, 3, 16], 20: [0, 3, 2, 13, 4, 7, 6, 1, 8, 11, 10, 5, 12, 15, 14, 9, 0, 3, 2, 13]
seed, a, c, m, n = [0, 1103515245, 12345, 4294967296], 20: [0, 12345, 3554416254, 2802067423, 3596950572, 229283573, 3256818826, 1051550459, 3441282840, 2941955441, 551188310, 2951033815, 1772930244, 2518396845, 639546082, 1381971571, 1695770928, 2121308585, 3866696494, 3144468175]
seed, a, c, m, n = [1, 2, 0, 19], 20: [1, 2, 4, 8, 16, 13, 7, 14, 9, 18, 17, 15, 11, 3, 6, 12, 5, 10, 1, 2]
seed, a, c, m, n = [1, 32, 0, 19], 20: [1, 13, 17, 12, 4, 14, 11, 10, 16, 18, 6, 2, 7, 15, 5, 8, 9, 3, 1, 13]
seed, a, c, m, n = [1, 16807, 0, 2147483647], 20: [1, 16807, 282475249, 1

### Método de Monte Carlo

In [4]:
def monteCarlo(f: callable, a: float, b: float, n: int) -> float:
    """
    Estimate the integral of f from a to b using the Monte Carlo method.
    a, b: interval [a, b]
    b is inf. if b is None
    """
    def estimate(f: callable, a: float, b: float, n: int) -> float:
        """
        Do the estimation method for fixed interval [a, b]
        """
        r = 0
        for i in range(n):
            x = rnd.random()
            r += f(a + (b - a) * x)
        return (b - a) * r / n

    g = lambda x: f(1/x - 1) / x**2
    match b:
        case None: return estimate(g, 0, 1, n) + estimate(f, a, 0, n)
        case _: return estimate(f, a, b, n)

def monteCarlo_multiple(f: callable, I: list[tuple[float, float]], n: int) -> float:
    """
    Estimate the multiple integral of function f over intervals in I using the Monte Carlo method.
    I[i][0], I[i][1]: interval [a, b]
        b is inf. if b is None => a must be 0
    f: function to integrate, takes a list of variables
    """
    def estimate(f: callable, I: list[tuple[float, float]], n: int) -> float:
        """
        Do the estimation method for fixed intervals
        """
        r = 0
        for i in range(n):
            x = [rnd.random() for _ in range(len(I))]
            r += f(x, I)
        m = 1
        for i in range(len(I)): m *= I[i][1] - I[i][0] if I[i][1] is not None else 1
        return m * r / n

    def g(x: list[float], I: list[tuple[float, float]]) -> float:
        """
        Function to estimate with fixed intervals [a, b]
        """
        denominator = 1
        for i in range(len(I)):
            if I[i][1] is None:
                assert I[i][0] == 0
                denominator *= x[i]**2
                x[i] = 1/x[i] - 1
            else:
                x[i] = I[i][0] + (I[i][1] - I[i][0]) * x[i]
        return f(x) / denominator

    return estimate(g, I, n)

## Ejercicio 1

### Item (a)

In [5]:
n = 10
seeds = [3792, 1004, 2100, 1234]
for seed in seeds:
    print(vonNeumann(seed, n))

[3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792, 3792]
[1004, 80, 64, 40, 16, 2, 0, 0, 0, 0]
[2100, 4100, 8100, 6100, 2100, 4100, 8100, 6100, 2100, 4100]
[1234, 5227, 3215, 3362, 3030, 1809, 2724, 4201, 6484, 422]


### Item (b)

In [6]:
a = 5
c = 4
m = 2**5
n = 10
seed_list = [4, 50]
for seed in seed_list:
    print(f"CASO con seed = {seed}")
    print(congruentialGenerator(seed, a, c, m, n))
    print(period_congruentialGenerator(seed, a, c, m))

CASO con seed = 4
[4, 24, 28, 16, 20, 8, 12, 0, 4, 24]
8
CASO con seed = 50
[50, 30, 26, 6, 2, 14, 10, 22, 18, 30]
8


### Item (c)

In [7]:
cases = [
    [None, 125, 3, 2**9],
    [None, 123, 3, 2**9],
    [None, 5, 0, 71],
    [None, 7, 0, 71]
]

for seed, a, c, m in cases:
    period = period_congruentialGenerator(seed, a, c, m)
    print(f"El caso: a, c, m = {a}, {c}, {m} tiene período = {period}")
    print(f" Luego, el período {'no' if period != m - (c == 0) else ''} es máximo.")

El caso: a, c, m = 125, 3, 512 tiene período = 512
 Luego, el período  es máximo.
El caso: a, c, m = 123, 3, 512 tiene período = 256
 Luego, el período no es máximo.
El caso: a, c, m = 5, 0, 71 tiene período = 5
 Luego, el período no es máximo.
El caso: a, c, m = 7, 0, 71 tiene período = 70
 Luego, el período  es máximo.


## Ejercicio 2

In [8]:
def ej2_game() -> bool:
    """
    Simulate the game
    """
    u = rnd.random()
    n = 2 if u < 0.5 else 3
    sw = sum([rnd.random() for i in range(n)])
    return sw >= 1

def ej2_simulation(n: int) -> float:
    cntWin = 0
    for i in range(n):
        cntWin += ej2_game()
    return cntWin / n

n_list = [10**2, 10**3, 10**4, 10**5, 10**6]
print(f"El valor real de la probabilidad teórica es: {2/3}")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    print(f"Para n = {n}: {ej2_simulation(n)}")

El valor real de la probabilidad teórica es: 0.6666666666666666
Los valores de las probabilidades simuladas son:
Para n = 100: 0.7
Para n = 1000: 0.626
Para n = 10000: 0.663
Para n = 100000: 0.66696
Para n = 1000000: 0.667077


## Ejercicio 3

In [9]:
def ej3_game() -> bool:
    """
    Simulate the game
    """
    u = rnd.random()
    n = 2 if u < 1/3 else 3
    sw = sum([rnd.random() for i in range(n)])
    return sw <= 2

def ej3_simulation(n: int) -> float:
    cntWin = 0
    for i in range(n):
        cntWin += ej3_game()
    return cntWin / n

n_list = [10**2, 10**3, 10**4, 10**5, 10**6]
print(f"El valor real de la probabilidad teórica es: {8/9}")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    print(f"Para n = {n}: {ej3_simulation(n)}")

El valor real de la probabilidad teórica es: 0.8888888888888888
Los valores de las probabilidades simuladas son:
Para n = 100: 0.95
Para n = 1000: 0.894
Para n = 10000: 0.889
Para n = 100000: 0.88981
Para n = 1000000: 0.889671


## Ejercicio 4

In [10]:
from scipy.stats import expon

def ej4_simulation_a(n: int, m: list[float], p: list[float]) -> float:
    cntWin = 0
    time_list = [expon.rvs(scale=m[i], size=n) for i in range(3)]
    idx = [0, 0, 0]
    for i in range(n):
        c = rnd.random(); i = 0
        while c-p[i] > 0: c -= p[i]; i += 1
        time = time_list[i][idx[i]]; idx[i] += 1
        cntWin += time <= 4
    return cntWin / n

def ej4_simulation_b(n: int, m: list[float], p: list[float]) -> float:
    cntWin = [0, 0, 0]
    idx = [0, 0, 0]
    time_list = [expon.rvs(scale=m[i], size=n) for i in range(3)]
    for i in range(n):
        c = rnd.random(); i = 0
        while c-p[i] > 0: c -= p[i]; i += 1
        time = time_list[i][idx[i]]; idx[i] += 1
        if time >= 4: cntWin[i] += 1
    prob_ge_4 = 1 - ej4_simulation_a(n, m, p)
    for i in range(3): cntWin[i] /= prob_ge_4 * n
    return cntWin


n = 10**6
m = [3, 4, 5]
p = [0.4, 0.32, 0.28]

print(f"La probabilidad de que un cliente espere menos de 4 minutos es: {ej4_simulation_a(n, m, p)}")
print(f"Las probabilidades de elección de cada caja dado que el cliente esperó más de 4 minutos son: {ej4_simulation_b(n, m, p)}")

La probabilidad de que un cliente espere menos de 4 minutos es: 0.65114
Las probabilidades de elección de cada caja dado que el cliente esperó más de 4 minutos son: [0.3025600553587309, 0.3351547957074108, 0.3600404896502621]


## Ejercicio 5

In [11]:
n_list = [10**i for i in range(2, 7)]

# Item a: int_0^1 (1-x^2)^{3/2} dx = 3*pi/16 = 0.589
print(f"Vemos la integral: int_0^1 (1-x^2)^{{3/2}} dx = 0.589, y los valores de la simulación son:")
f = lambda x: (1-x**2)**(3/2)
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo(f, 0, 1, n)}")
print()

# Item b: int_2^3 x/(x^2-1)dx = 1/2 log(8/3) = 0.49
print(f"Vemos la integral: int_2^3 x/(x^2-1) dx = 0.49, y los valores de la simulación son:")
f = lambda x: x/(x**2-1)
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo(f, 2, 3, n)}")
print()

# Item c: int_0^inf x*(1+x^2)^{-2} dx = 0.5
print(f"Vemos la integral: int_0^inf x*(1+x^2)^{{-2}} dx = 0.5, y los valores de la simulación son:")
f = lambda x: x*(1+x**2)**(-2)
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo(f, 0, None, n)}")
print()

# Item d: int_{-inf}^inf e^{-x^2} dx = sqrt(pi) = 1.77
print(f"Vemos la integral: int_{{-inf}}^inf e^{{-x^2}} dx = 1.77, y los valores de la simulación son:")
print(f"    Como es una funcion par, calculamos la integral en [0, inf] y la multiplicamos por 2")
f = lambda x: math.exp(-x**2)
for n in n_list:
    print(f"    Para n = {n}: {2*monteCarlo(f, 0, None, n)}")
print()

# Item e: int_0^1 int_0^1 e^{(x+y)^2} dx dy = 4.8991
print(f"Vemos la integral: int_0^1 int_0^1 e^{{(x+y)^2}} dx dy = 4.8991, y los valores de la simulación son:")
f = lambda x: math.exp((x[0]+x[1])**2)
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo_multiple(f, [(0, 1), (0, 1)], n)}")
print()

# Item f: int_0^inf int_0^x e^{-(x+y)} dy dx = 0.5
print(f"Vemos la integral: int_0^inf int_0^x e^{{-(x+y)}} dy dx = 0.5, y los valores de la simulación son:")
print(f"    Consideramos int_0^inf int_0^inf g(x, y) dy dx con g(x) = e^{{-(x+y)}} if x > y, 0 otherwise")
f = lambda x: math.exp(-(x[0]+x[1])) if x[0] > x[1] else 0
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo_multiple(f, [(0, None), (0, None)], n)}")
print()

Vemos la integral: int_0^1 (1-x^2)^{3/2} dx = 0.589, y los valores de la simulación son:
    Para n = 100: 0.6141645800312137
    Para n = 1000: 0.5955490078453325
    Para n = 10000: 0.5897399198517184
    Para n = 100000: 0.5901210189407929
    Para n = 1000000: 0.5887392685297388

Vemos la integral: int_2^3 x/(x^2-1) dx = 0.49, y los valores de la simulación son:
    Para n = 100: 0.48723193916528
    Para n = 1000: 0.4922942381054338
    Para n = 10000: 0.4898750089632712
    Para n = 100000: 0.4906944319406067
    Para n = 1000000: 0.4902770934331875

Vemos la integral: int_0^inf x*(1+x^2)^{-2} dx = 0.5, y los valores de la simulación son:
    Para n = 100: 0.4913457845952184
    Para n = 1000: 0.48757411565381054
    Para n = 10000: 0.5013399466800501
    Para n = 100000: 0.50019740912655
    Para n = 1000000: 0.4996790167081169

Vemos la integral: int_{-inf}^inf e^{-x^2} dx = 1.77, y los valores de la simulación son:
    Como es una funcion par, calculamos la integral en [0, inf

## Ejercicio 6

In [12]:
print(f"Vamos a considerar la función f(x, y) = 1 si x^2 + y^2 <= 1, 0 otherwise")
n_list = [10**i for i in range(3, 6)]
f = lambda x: 1 if x[0]**2 + x[1]**2 <= 1 else 0
print(f"La integral de f(x, y) en el círculo de radio 1 es pi = {math.pi}")
print(f"Los valores de la simulación son:")
for n in n_list:
    print(f"    Para n = {n}: {monteCarlo_multiple(f, [(-1, 1), (-1, 1)], n)}")

Vamos a considerar la función f(x, y) = 1 si x^2 + y^2 <= 1, 0 otherwise
La integral de f(x, y) en el círculo de radio 1 es pi = 3.141592653589793
Los valores de la simulación son:
    Para n = 1000: 3.076
    Para n = 10000: 3.132
    Para n = 100000: 3.14412


## Ejercicio 7

In [13]:
def ej7_simulation(n: int) -> float:
    expected = 0
    for i in range(n):
        s = 0
        cntWin = 0
        while s < 1:
            s += rnd.random()
            cntWin += 1
        expected += cntWin
    return expected / n

n_list = [10**i for i in range(2, 7)]
print(f"El valor real de la probabilidad teórica es: e = {math.e}")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    print(f"    Para n = {n}: {ej7_simulation(n)}")

El valor real de la probabilidad teórica es: e = 2.718281828459045
Los valores de las probabilidades simuladas son:
    Para n = 100: 2.78
    Para n = 1000: 2.717
    Para n = 10000: 2.7169
    Para n = 100000: 2.7217
    Para n = 1000000: 2.719051


## Ejercicio 8

In [14]:
# ITEM A
def ej8_simulation_a(n: int) -> float:
    expected = 0
    for i in range(n):
        m = 1
        cntWin = 0
        limit = math.exp(-3)
        while m >= limit:
            m *= rnd.random()
            cntWin += 1
        expected += (cntWin - 1)
    return expected / n

n_list = [10**i for i in range(2, 7)]
print(f"Item A:")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    print(f"    Para n = {n}: {ej8_simulation_a(n)}")
print()

# ITEM B
def ej8_simulation_b(n: int) -> list[float]:
    r = dict()
    for i in range(n):
        m = 1
        cntWin = 0
        limit = math.exp(-3)
        while m >= limit:
            m *= rnd.random()
            cntWin += 1
        cntWin -= 1
        r[cntWin] = r.get(cntWin, 0) + 1
    for x in r: r[x] /= n
    return r

n_list = [10**6]
print(f"Item B:")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    prob_list = ej8_simulation_b(n)
    print(f"    Para n = {n}: {[prob_list.get(i, 0) for i in range(0, 7)]}")

Item A:
Los valores de las probabilidades simuladas son:
    Para n = 100: 3.08
    Para n = 1000: 2.965
    Para n = 10000: 3.0089
    Para n = 100000: 2.99853
    Para n = 1000000: 2.999525

Item B:
Los valores de las probabilidades simuladas son:
    Para n = 1000000: [0.050123, 0.149753, 0.223808, 0.223521, 0.168091, 0.100784, 0.050329]


## Ejercicio 9

In [15]:
def ej9_simulation(n: int) -> float:
    cntWin = 0
    for i in range(n):
        x = rnd.randint(1, 6)
        s = 0
        if x in [1, 6]: s = 2 * rnd.randint(1, 6)
        else: s = sum([rnd.randint(1, 6) for i in range(2)])
        cntWin += s > 6
    return cntWin / n

n_list = [10**i for i in range(2, 7)]
print(f"El valor real de la probabilidad teórica es: ?")
print(f"Los valores de las probabilidades simuladas son:")
for n in n_list:
    print(f"    Para n = {n}: {ej9_simulation(n)}")

El valor real de la probabilidad teórica es: ?
Los valores de las probabilidades simuladas son:
    Para n = 100: 0.5
    Para n = 1000: 0.58
    Para n = 10000: 0.5579
    Para n = 100000: 0.55286
    Para n = 1000000: 0.555012
